In [ ]:
#@title Environment Setup

from pathlib import Path

OPTIONS = {}

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = 'ComfyUI'
OPTIONS['USE_GOOGLE_DRIVE'] = USE_GOOGLE_DRIVE
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI

if OPTIONS['USE_GOOGLE_DRIVE']:
    !echo "Mounting Google Drive..."
    %cd /

    from google.colab import drive
    drive.mount('/content/drive')

    WORKSPACE = "/content/drive/MyDrive/ComfyUI"
    %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup ComfyUI =- && git clone https://github.com/comfyanonymous/ComfyUI
%cd $WORKSPACE

if OPTIONS['UPDATE_COMFY_UI']:
  !echo -= Updating ComfyUI =-
  !git pull

!echo -= Install dependencies =-
!pip install xformers!=0.0.18 -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu118 --extra-index-url https://download.pytorch.org/whl/cu117

In [ ]:
%cd {repo_dir}/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
!git clone https://github.com/WASasquatch/was-node-suite-comfyui.git
!git clone https://github.com/Derfuu/Derfuu_ComfyUI_ModdedNodes.git
!git clone https://github.com/EllangoK/ComfyUI-post-processing-nodes.git
!git clone https://github.com/ssitu/ComfyUI_UltimateSDUpscale --recursive
!git clone https://github.com/BlenderNeko/ComfyUI_TiledKSampler.git
!git clone https://github.com/BlenderNeko/ComfyUI_ADV_CLIP_emb.git
!git clone https://github.com/failfa-st/failfast-comfyui-extensions.git
!git clone https://github.com/LucianoCirino/efficiency-nodes-comfyui.git
!git clone https://github.com/RockOfFire/ComfyUI_Comfyroll_CustomNodes.git
!git clone https://github.com/ltdrdata/ComfyUI-Impact-Pack.git
!git clone https://github.com/SeargeDP/SeargeSDXL.git
!git clone https://github.com/BlenderNeko/ComfyUI_Cutoff
!git clone https://github.com/LucianoCirino/efficiency-nodes-comfyui
!git clone https://github.com/Davemane42/ComfyUI_Dave_CustomNode
!git clone https://github.com/pythongosssss/ComfyUI-WD14-Tagger.git
!git clone https://github.com/ssitu/ComfyUI_restart_sampling
!git clone https://github.com/pythongosssss/ComfyUI-Custom-Scripts
!git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors.git
!git clone https://github.com/twri/sdxl_prompt_styler.git
!git clone https://github.com/Ser-Hilary/SDXL_sizing.git
!git clone https://github.com/laksjdjf/IPAdapter-ComfyUI.git
!wget https://huggingface.co/faisalhr1997/comfyui/resolve/main/SDXLAspectRatio.py
!wget https://huggingface.co/faisalhr1997/comfyui/resolve/main/ksampler_sdxl.py
!git clone https://github.com/JPS-GER/ComfyUI_JPS-Nodes.git

!python /content/ComfyUI/custom_nodes/comfy_controlnet_preprocessors/install.py
!pip install -r {repo_dir}/custom_nodes/ComfyUI-WD14-Tagger/requirements.txt

In [ ]:
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors -P ./models/checkpoints/
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors -P ./models/checkpoints/
!wget -c https://civitai.com/api/download/models/156005 -P ./models/checkpoints/
!wget -c https://huggingface.co/madebyollin/sdxl-vae-fp16-fix/resolve/main/sdxl_vae.safetensors -P ./models/vae/
!wget -c https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors -P ./models/loras/ #SDXL offset noise lora
!wget -c https://huggingface.co/diffusers/controlnet-canny-sdxl-1.0/resolve/main/diffusion_pytorch_model.fp16.safetensors -P ./models/controlnet/
!wget -c https://huggingface.co/diffusers/controlnet-depth-sdxl-1.0/resolve/main/diffusion_pytorch_model.fp16.bin -P ./models/controlnet/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.bin -P ./models/
!wget -c https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/image_encoder/pytorch_model.bin -P ./models/clip_vision/
!wget -c https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P ./models/upscale_models/
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P ./models/upscale_models/
!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P ./models/upscale_models/

In [ ]:
# @title 3. Run ```ComfyUI``` [*](https://huggingface.co/faisalhr1997/comfyui/resolve/main/2workflow.json)



import os
from IPython.display import clear_output
from google.colab import drive


launch_arguments = '--highvram --dont-print-server --use-pytorch-cross-attention --dont-upcast-attention --enable-cors-header' # @param {type:"string"}
drive_output = True # @param {type:"boolean"}
tunnel = "Cloudflared" # @param ["Cloudflared", "Localtunnel", "Iframe"]

if drive_output:
    drive.mount('/content/drive')
    %cd /content/drive/MyDrive
    !mkdir comfyui_output
    launch_arguments += " --output-directory /content/drive/MyDrive/comfyui_output"

%cd {repo_dir}

if tunnel == "Cloudflared":

  !wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64
  import atexit, requests, subprocess, time, re
  from random import randint
  from threading import Timer
  from queue import Queue
  def cloudflared(port, metrics_port, output_queue):
      atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
      attempts, tunnel_url = 0, None
      while attempts < 10 and not tunnel_url:
          attempts += 1
          time.sleep(3)
          try:
              tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
          except:
              pass
      if not tunnel_url:
          raise Exception("Can't connect to Cloudflare Edge")
      output_queue.put(tunnel_url)
  output_queue, metrics_port = Queue(), randint(8100, 9000)
  thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
  thread.start()
  thread.join()
  tunnel_url = output_queue.get()
  os.environ['webui_url'] = tunnel_url
  clear_output()
  print(f"\033[31m>> link:\033[0m")
  print(tunnel_url)
  print("\n\n")

  !python main.py {launch_arguments}

elif tunnel == "Localtunnel":

  !npm install -g localtunnel

  import subprocess
  import threading
  import time
  import socket
  import urllib.request

  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")
    clear_output()
    print("\n\n")
    print(">> The password/enpoint ip for localtunnel is:\n", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
    print("\n\n")
    print(f"\033[31m>> link:\033[0m")
    p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
    for line in p.stdout:
      print(line.decode(), end='')


  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

  !python main.py {launch_arguments}

elif tunnel == "Iframe":
  import threading
  import time
  import socket
  def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
          break
        sock.close()
    from google.colab import output
    output.serve_kernel_port_as_iframe(port, height=1024)
    print("to open it in a window you can open this link here:")
    output.serve_kernel_port_as_window(port)

  threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

  !python main.py {launch_arguments}